# **Ising Model in 2D**

<i class="fa fa-home fa-2x"></i><a href="../index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/master/notebook/statistical-mechanics/ising_model.ipynb

The Cython computing kernel is adapted from the code presented in the 
[Pythonic Perambulations](https://jakevdp.github.io/blog/2017/12/11/live-coding-cython-ising-model) blog.

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## Goals

* Understand how the Ising model can predict the ferromagnetic or antiferromagnetic behavior of a square (2D) spin lattice.
* Compute the average magnetization of the Ising model using a Monte Carlo algorithm.
* Examine the effects of the interaction parameter $J$ (ferromagnetic vs. antiferromagnetic behavior).
* Identify the critical temperature of the system from the simulation results.
* Investigate finite-size effects and their effect on fluctuations.

# **Background Theory**

[More in the background theory](./theory/theory_ising_model.ipynb)

## **Tasks and exercises**

1. Run the simulation with the default parameters (and $J>0$) and check the plot of the magnetization per spin as a function of the simulation step. What happens at very low temperatures? And at intermediate temperatures? And at very high temperatures? And what happens for $J<0$?

    <details>
    <summary style="color: red">Solution</summary>
  For a positive $J$ value, parallel neighboring spins are energetically favored.
  Therefore, at $T=0$ we expect all spins to be aligned: the system is ferromagnetic.
  At low temperature, spin will still tend to be aligned with their neighbors, with some
  local fluctuations as temperature increases.
  Fluctuations become very large close to $T_c$, and above $T_c$
  they become predominant and, on average, we obtain zero net magnetization.
  On the other hand, $J<0$ will lead to a checkerboard pattern for the final spin 
  configuration at low temperatures, i.e., an antiferromagnetic configuration.
    When $J=0$, there is no interaction between spins and we always obtain a random spin configuration.<br><br>
    </details>

2. Simulate a large system (200x200) with, e.g., $J=1$ and $T=2.5$. Is the simulation converging with the selected number of steps? Which values can you inspect to determine the convergence of the simulation?

    <details>
    <summary style="color: red">Solution</summary>
  A large number of simulation steps can be needed before properties converge, especially when we are close to $T_C$.<br>
  
   You can check the plot of the integrated quantities (total energy, magnetization per spin) to see if the value is converged (their values are constant save for small fluctuations with simulation step), or if you need to increase the number of steps.<br><br>
    </details>

3. Set $J=1$ and start from all spin up (i.e., disable a random starting configuration). Run multiple simulations at various temperatures. Can you approximately identify the critical (Curie) temperature using the plot below the interactive controls? 

  <details>
   <summary style="color: red">Solution</summary>
  We suggest that you start from a small size system (50x50) to avoid excessively long simulations, and choose a long number of simulation steps (1000) to be approximately converged (see previous question).<br>

  Theoretically, $T_C$ for the 2D Ising model (and $J=1$) is about 2.27 (see the <a href="https://en.wikipedia.org/wiki/Square_lattice_Ising_model">Wikipedia page of the Square lattice Ising model</a>), as visualized by the analytical exact solution plotted below the interactive controls.<br>
    
  You should see that the simulation results follow relatively closely the analytical curve for $T \ll T_c$ or for $T \gg T_C$, but large fluctuations occur close to the Curie temperature. We suggest that you run multiple runs for each temperature to see the fluctuations. You should be able to reproduce the curve and thus approximately identify the transition temperature.<br>
    
  We suggest that you try the same with different values of $J$.<br><br>
  </details>

4. Set $J=1$, and simulate the system close to $T_C$. Investigate the magnitude of fluctuations as a function of system size.

  <details>
    <summary style="color: red">Solution</summary>
  Consider for instance both a small (50x50) and a large system (200x200), $J=1$, and $T=2.5$. Run a few simulations (at least 5 or 10) for each system size (note that each simulation for the large system will take many seconds). Verify that fluctuations are less pronounced for the larger system.<br>
    
  Note that the difference in fluctuations might not be very large, since the two systems are still relatively similar in size. The difference in the magnitude of the fluctuation can also be visualized by inspecting the fluctuations of the averaged quantities in the two plots of the magnetization per spin and total energy as a function of the simulation step.
  </details>

5. Consider a 100x100 system, set $J=1$ and $T=1.7$. Use a large number of simulation steps, and enable the randomization of the initial spin configuration. Investigate the formation of domains. Are they more stable for large or small systems?

  <details>
    <summary style="color: red">Solution</summary>
  If you run the simulation multiple times, you will notice that often the simulation does not reach the expected (positive or negative) analytical exact result, but will have intermediate magnetization values due to the formation of domains. These might disappear during the simulation, but sometimes they will remain even after 1000 simulation steps.<br>
    
  If you consider a smaller system (e.g. 50x50), you should notice that there is a higher probability that the domains disappear during the simulation. In fact, the cost of a domain scales as the length of its boundary, which (for very large systems) becomes a negligible cost with respect to the total energy of the system (that scales with the number of spins, i.e. quadratically with respect to the system size) and therefore can exist for a long time; in addition, domains will have a higher probability to merge for a small system.
  </details>

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

In [ ]:
%reload_ext Cython
%matplotlib widget

In [ ]:
import numpy as np
from ipywidgets import interact, FloatSlider, Button, Output, IntSlider, VBox
from ipywidgets import HBox, Checkbox, IntProgress, HTML
import matplotlib.pyplot as plt
from time import sleep
import matplotlib.gridspec as gridspec
from scipy.ndimage import convolve
from matplotlib.animation import FuncAnimation
import base64

In [ ]:
def random_spin_field(N, M):
    """Randomize the initial spin configuration."""
    return np.random.choice([-1, 1], size=(N, M))

def all_up_spin_field(N, M):
    """Set all spin up."""
    return np.ones((N, M), dtype=int)

run_button = Button(description='Run simulation')
run_button.style.button_color = 'lightgreen'
play_button = Button(description='Play', disabled=True)

random_checkbox = Checkbox(value=False, description="Randomize initial spin configuration", style={'description_width': 'initial'})

jvalue_slider = FloatSlider(value = 1.0, min = -2.0, max = 2.0, description = 'Exchange interaction J',
                    style={'description_width': 'initial'}, continuous_update=False)
num_slider = IntSlider(value=100, min=50, max=200, step=10, description="Size", continuous_update=False)
temp_slider = FloatSlider(value=2, min=0.5, max=4, step=0.1, description="Temperature", continuous_update=False) # Units of J/k
step_slider = IntSlider(value=100, min=100, max=1000, step=50, description="Num steps", continuous_update=False)
frame_slider = IntSlider(value=0, min=0, max=step_slider.value, description="Frame", layout={'width':'800px'}, disabled=True)

In [ ]:
%%cython

cimport cython

import numpy as np
cimport numpy as np

from libc.math cimport exp
from libc.stdlib cimport rand
cdef extern from "limits.h":
    int RAND_MAX

@cython.boundscheck(False)
@cython.wraparound(False)
def cy_ising_step(np.int64_t[:, :] field, float beta, float J):
    """Update the Ising step, each step actually contains N*M steps.
       
    Args:
        field: matrix representation for the spin configuration.
        beta: 1/kbT
        J: the strength of exchange interaction.
        
    Returns:
        New spin configuration and the total energy change.
    """
    cdef int N = field.shape[0]
    cdef int M = field.shape[1]
    cdef int i
    cdef np.ndarray x, y
    cdef float dE = 0.0
    
    x = np.random.randint(N, size=(N*M))
    y = np.random.randint(M, size=(N*M))
    
    for i in range(N*M):
        dE += _cy_ising_update(field, x[i], y[i], beta, J)
        
    return np.array(field), dE

@cython.boundscheck(False)
@cython.wraparound(False)
cdef _cy_ising_update(np.int64_t[:, :] field, int n, int m, float beta, float J):
    """Monte Carlo simulation using the Metropolis algorithm.
    
    Args:
        field: matrix representation for the spin configuration.
        n: chosen row index.
        m: chosen column index.
        beta: 1/kbT
        J: the strength of exchange interaction.
        
    Returns:
        The total energy change.
    """
    cdef int total
    cdef int N = field.shape[0]
    cdef int M = field.shape[1]

    total = field[(n+1)%N, m] + field[n, (m+1)%M] + field[(n-1)%N, m] + field[n, (m-1)%M]
    cdef float dE = 2.0 * J * field[n, m] * total
    if dE <= 0:
        field[n, m] *= -1
        return dE
    elif exp(-dE * beta) * RAND_MAX > rand():
        field[n, m] *= -1
        return dE
    else:
        return 0

In [ ]:
pause = True;

def on_frame_change(b):
    """Update the plot for playing the animation."""
    global fig, v1
    fig.set_data(images[frame_slider.value])
    v1.set_data([frame_slider.value, frame_slider.value],[-1.1, 1.1])
    v2.set_data([frame_slider.value, frame_slider.value],[-5000, 5000])

    
def compute_total_energy(M, J):
    """Compute the total energy of the given spin configuration."""
    a = np.ones(np.shape(M));
    c = convolve(M, a, mode='constant')
    c = (c-M)*M*J
    return c.sum()


def update(frame):
    """Update function for the animation."""
    global pause
    
    if pause:
        ani.event_source.stop()
    else:
        frame_slider.value = frame
        return (fig)
    
def play_animation(event):
    """OnClick function the 'Play' button."""
    global pause
    
    pause ^= True
    if play_button.description == "Pause":
        play_button.description = "Play"
        ani.event_source.stop()
    else:
        play_button.description = "Pause"
        ani.event_source.start()
    

frame_slider.observe(on_frame_change, names='value')

images = [all_up_spin_field(num_slider.value, num_slider.value)]

img = plt.figure(tight_layout=True, figsize=(8,5))
img.canvas.header_visible = False
gs = gridspec.GridSpec(4, 2)

ax1 = img.add_subplot(gs[:, 0])
ax2 = img.add_subplot(gs[0:2, 1])
ax3 = img.add_subplot(gs[2:4, 1])

fig = ax1.imshow(images[0], vmin=-1, vmax=1)
ax1.axes.xaxis.set_ticklabels([])
ax1.axes.yaxis.set_ticklabels([])
ax1.set_title('Spin up (yellow), spin down (purple)', fontsize=12)

line1, = ax2.plot([0], [0], 'r-')
line2, = ax3.plot([0], [0], 'r-')
v1 = ax2.axvline(x=0, c='black')
v2 = ax3.axvline(x=0, c='black')

ax2.set_xlim([0, step_slider.value])
ax2.set_ylim([-1.1, 1.1])
ax2.set_title('Magnetization per spin', fontsize=12)

ax3.set_xlim([0, step_slider.value])
ax3.set_ylim([-1, 1])
ax3.set_title(r'Total energy (E$_{init}$=0)', fontsize=12)
ax3.set_xlabel('Step', fontsize=12)

ani = FuncAnimation(img, update, interval= 20, frames=np.arange(0, step_slider.value+1), blit=True)

display(frame_slider,
    HBox([num_slider, step_slider]),
    HBox([jvalue_slider, temp_slider]),
    HBox([random_checkbox, run_button, play_button]))

In [ ]:
def get_analytical_plot_data(J, k_B=1):
    """Exact solution of the 2D square Ising model.
    
    See e.g.: https://en.wikipedia.org/wiki/Square_lattice_Ising_model
    """
    temperature = np.linspace(0.01, 4, 500)
    magnetization = np.zeros(len(temperature))
    Tc = 2 * J / k_B / np.log(1 + np.sqrt(2))
    magnetization[temperature < Tc] = (1-1./(np.sinh(2 * J / k_B / temperature[temperature < Tc])**4))**(1/8)

    return temperature, magnetization

def reset_mag_plot(ax):
    """Reset the magnetization plot.
    
    Clear the axes and re-draw the analytical exact solution.
    """
    global ax_mag, random_checkbox
    temps, magnetization = get_analytical_plot_data(J=jvalue_slider.value, k_B=1)
    
    ax.clear()
    ax.plot(temps, magnetization, 'b-')
    if random_checkbox.value:
        ax.plot(temps, -magnetization, 'r-')        
    ax.set_xlabel("$T$")
    ax.set_ylabel(r"$\langle \sigma \rangle$")
    ax.set_title("Magnetization per site vs. temperature")
    ax.set_xlim((0, 4))

def create_mag_plot():
    """Create the magnetization plot.
    
    To be called only one to create it and return the axes object.
    """
    img_mag = plt.figure(tight_layout=True, figsize=(5,3.5))
    img_mag.canvas.header_visible = False
    gs_mag = gridspec.GridSpec(1, 1)
    ax_mag = img_mag.add_subplot(gs_mag[:, :])

    reset_mag_plot(ax_mag)
    
    return ax_mag    

# Create and display the magnetization plot.
ax_mag = create_mag_plot()

In [ ]:
def run_simulation(b):
    """Callback to be called when the 'Run simulation' button is clicked.
    
    Takes care of temporarily disabling the button, running the simulation and updating
    the various plots, and re-enabling the button again.
    """
    global ax_mag

    play_button.disabled = True
    run_button.disabled = True
    frame_slider.disabled = True
    run_button.style.button_color = 'red'
    global images, fig
    if random_checkbox.value:
        images = [random_spin_field(num_slider.value, num_slider.value)]
    else:
        images = [all_up_spin_field(num_slider.value, num_slider.value)]
    
    x = np.arange(step_slider.value + 1)
    y1 = []
    y2 = [0]

    for i in range(step_slider.value):
        imag, dE = cy_ising_step(images[-1].copy(), beta=1.0/temp_slider.value, J=jvalue_slider.value)
        images.append(imag)
        y2.append(dE+y2[-1])

    frame_slider.max = step_slider.value
    ax2.set_xlim([0, step_slider.value])

    fig.set_data(images[frame_slider.max - 1])

    for i in images:
        y1.append(i.sum()*1.0/(num_slider.value * num_slider.value))

    y1 = np.array(y1)
    y2 = np.array(y2)

    line1.set_data(x, y1)
    line2.set_data(x, y2)

    ax3.set_ylim([y2.min(), y2.max()])
    ax3.set_xlim([0, step_slider.value])

    frame_slider.value = frame_slider.max

    ani.frames = np.arange(0, step_slider.value+1)
    ani._iter_gen = lambda: iter(ani.frames)
    ani.save_count = len(ani.frames)
    ani.frame_seq = ani.new_frame_seq()

    ax_mag.errorbar([temp_slider.value], [y1[-1]], fmt='.g')
    
    frame_slider.disabled = False
    play_button.disabled = False
    run_button.disabled = False
    run_button.style.button_color = 'lightgreen'
    return y1

def on_needs_reset(b):
    """Callback to be called when the magnetization plot needs to be cleared and reset."""
    global ax_mag
    reset_mag_plot(ax_mag)

# Attach actions to the buttons
run_button.on_click(run_simulation)
play_button.on_click(play_animation)
# Attach reset actions (for the magnetization plot) when some of the
# simulation parameters are changed (all except the temperature)
jvalue_slider.observe(on_needs_reset, names='value')
num_slider.observe(on_needs_reset, names='value')
step_slider.observe(on_needs_reset, names='value')
random_checkbox.observe(on_needs_reset, names='value')

## Legend

(How to use the interactive visualization)

### Controls

The "size" slider defines the number of spins along each dimension.
You can also adapt the number of simulation steps, the value of the
exchange interaction parameter $J$, and the temperature of the simulation
(note: units have been chosen so that the Boltzmann constant $k_B=1$).

By default, we set all spin up for the initial configuration.
This induces a bias for short simulations (but can help avoid the formation of
domains); you can instead randomize the initial configuration by ticking
the checkbox "Randomize initial spin configuration".

### Running the simulation and output plots
Click the "Run simulation" button to start the simulation with the selected parameters.

In particular, the figures above the controls display the time evolution of the last simulation
that was executed.
The figure on the left shows the spin configuration. A yellow pixel represents 
a spin up and blue represents a spin down.
After the simulation finished, you can click "Play" to view the evolution of the
simulation step by step (or drag manually the "Frame" slider).
The figure on the top right shows the evolution of the magnetization per spin over
the simulation as a function of the Monte Carlo simulation step number.
The figure on the bottom right shows the total energy as a function of the step number.

In addition, at every new simulation, a new green point is added to the bottom plot,
showing the magnetization per spin as a function of temperature $T$.
The analytical exact solution for the chosen value of $J$ is also shown
(see [Wikipedia page on the square lattice Ising model](https://en.wikipedia.org/wiki/Square_lattice_Ising_model)
for more details on the exact solution). Note that when a random initial configuration is selected instead of all spin up, there shall be both blue and red curves for the analytical solution due to the possibility of having negative magnetization in that case.

Changing any slider (except for the temperature slider)
will reset the bottom plot.
You can change the temperature and run the simulation multiple times
to compare the analytical solution with the results of the simulation.